In [2]:
import warnings
warnings.filterwarnings('ignore')
from keras.optimizers import Nadam, Adam
from keras.layers import *
from keras.initializers import *
from keras.activations import softmax
from keras.layers import InputSpec, Layer, Input, Dense, merge, Conv1D
from keras.layers import Lambda, Activation, Dropout, Embedding, TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.regularizers import l2
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
import gc
import os
from keras.callbacks import *
from keras.optimizers import *
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,RepeatedKFold

Using TensorFlow backend.


In [3]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
target=train['target']

In [4]:
feature_topic_w2vdis=pd.read_hdf('feature_topic_w2vdis.h5', key='data')
feature_member_info=pd.read_hdf('feature_member_info_new.h5', key='data')[['salt','sex_count','look_freq_count','A_count','B_count','D_count']]
feature_invite_time_new=pd.read_hdf('feature_invite_time_new2.h5', key='data')
feature_uq_pagerank=pd.read_hdf('feature_uq_pagerank2.h5', key='data')
feature_today_invite=pd.read_hdf('feature_today_invite_new2.h5', key='data')
feature_uid_ctr_new=pd.read_hdf('feature_uid_ctr_new.h5', key='data')
feature_topic_count=pd.read_csv('feature_topic_count.csv')

train=pd.concat([train,
                 feature_topic_w2vdis.iloc[0:train.shape[0],:],
                 feature_member_info.iloc[0:train.shape[0],:],
                 feature_invite_time_new.iloc[0:train.shape[0],3:],
                 feature_uq_pagerank.iloc[0:train.shape[0],:],
                 feature_today_invite.iloc[0:train.shape[0],:],
                 feature_topic_count.iloc[0:train.shape[0],:],
                ],axis=1)
test=pd.concat([test,
                feature_topic_w2vdis.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_member_info.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_invite_time_new.iloc[train.shape[0]:,3:].reset_index(drop=True),
                feature_uq_pagerank.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_today_invite.iloc[train.shape[0]:,:].reset_index(drop=True),
                feature_topic_count.iloc[train.shape[0]:,:].reset_index(drop=True),
                ],axis=1)

In [5]:
feature_qid_freq=pd.read_hdf('feature_qid_freq2.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_qid_freq,how='left',on=['qid','uid'])
test=pd.merge(test,feature_qid_freq,how='left',on=['qid','uid'])
feature_uid_freq=pd.read_hdf('feature_uid_freq2.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_uid_freq,how='left',on=['qid','uid'])
test=pd.merge(test,feature_uid_freq,how='left',on=['qid','uid'])

In [6]:
feature_answer_base=pd.read_hdf('feature_answer_base.h5', key='data')
train=pd.merge(train,feature_answer_base,how='left',on=['qid','uid'])
test=pd.merge(test,feature_answer_base,how='left',on=['qid','uid'])
feature_Stime_history=pd.read_hdf('feature_Stime_history.h5', key='data').drop_duplicates(['qid','uid'])
train=pd.merge(train,feature_Stime_history,how='left',on=['qid','uid'])
test=pd.merge(test,feature_Stime_history,how='left',on=['qid','uid'])
feature_atime_new=pd.read_hdf('feature_atime_new.h5', key='data') 
train=pd.merge(train,feature_atime_new,how='left',on=['qid','uid'])
test=pd.merge(test,feature_atime_new,how='left',on=['qid','uid'])

In [10]:
def build_model():
    fea= Input(shape=(train.shape[1],))
    fea1 = Dense(1024, activation='relu')(fea)
    dense = Dropout(0.3)(fea1)
    dense = Dense(512, activation='relu')(dense)
    dense = Dropout(0.3)(dense)
    dense = Dense(512, activation='relu')(dense)
    dense = Dropout(0.3)(dense)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=[fea], outputs=pred)#
    model.compile(optimizer='adam', loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold

n_splits= 5
folds = StratifiedKFold(n_splits=n_splits)
sub1 = np.zeros((test.shape[0],1))
oof_pref1 = np.zeros((train.shape[0], 1))
count = 0

In [7]:
test = test.drop(['uid','qid','time'], axis=1)
train = train.drop(['target', 'uid','qid','time'], axis=1)

In [8]:
train_shape=train.shape

In [10]:
train.shape

(9489162, 122)

In [11]:
test.shape

(1141718, 122)

In [9]:
len(train)+len(test)

10630880

In [12]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mm = StandardScaler()
alldata = mm.fit_transform(pd.concat([train,test]).replace([np.inf, -np.inf], 0).fillna(0))

In [13]:
alldata

array([[-0.19652388, -0.14816786, -0.41910499, ..., -1.33222188,
        -0.55826741, -1.20170751],
       [ 0.98222788,  1.04079583,  0.15003249, ..., -1.33222188,
        -0.55826741, -1.20170751],
       [ 1.3073875 ,  1.41647053,  0.63756395, ...,  0.58949734,
         1.40293978,  0.9942194 ],
       ...,
       [ 0.68809863,  0.7632955 ,  0.15104319, ..., -0.56353419,
         0.74920405, -0.10374405],
       [-1.84760627, -0.49152014,  2.0443155 , ..., -1.33222188,
        -0.55826741, -1.20170751],
       [ 0.43056048,  0.40789563, -0.16890455, ..., -1.33222188,
        -0.55826741, -1.20170751]])

In [14]:
np.save("all_data_final.npy", alldata)

In [12]:
train = alldata[0:train_shape[0],:]#.reset_index(drop=True)
test = alldata[train_shape[0]:,:]#.reset_index(drop=True)

In [ ]:
from sklearn import metrics
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(oof_pref1, target)):
    print("FOLD_ ", count + 1)
    K.clear_session()
    filepath = "mlp%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=1, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=2, verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    model_nn =build_model()
    y_tr,y_vl=target[trn_idx], target[val_idx]
    train_fea_trn,train_fea_vl=train[trn_idx], train[val_idx]
    gc.collect()
    hist = model_nn.fit([train_fea_trn], y_tr, batch_size=1024, epochs=15, #
                        validation_data=([train_fea_vl ], y_vl),callbacks=callbacks, verbose=1, shuffle=True)#,,
    print('load_wight')
    model_nn.load_weights(filepath)
    print('start to predict')
    sub1 += model_nn.predict([test], batch_size=2048)
    oof_pref1[val_idx] = model_nn.predict([train_fea_vl], batch_size=2048)
    count += 1
    print('val_auc:',metrics.roc_auc_score(y_vl,oof_pref1[val_idx]))
    break

W1217 16:37:45.769602 140122212017920 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:92: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1217 16:37:45.770889 140122212017920 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1217 16:37:45.822329 140122212017920 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:99: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1217 16:37:45.823595 140122212017920 deprecation_wrapper.py:119] From /root/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:514: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeho

FOLD_  1


W1217 16:37:45.988299 140122212017920 deprecation.py:323] From /root/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7591329 samples, validate on 1897833 samples
Epoch 1/15
7591329/7591329 [==============================] - 68s 9us/step - loss: 0.3685 - acc: 0.8405 - val_loss: 0.3530 - val_acc: 0.8452

Epoch 00001: val_loss improved from inf to 0.35303, saving model to mlp0.h5
Epoch 2/15
4603904/7591329 [=================>............] - ETA: 24s - loss: 0.3537 - acc: 0.8466

In [12]:
#0.843927  0.34398 0.813170 0.03'uday_sametime_icount','uday_invite_rank','qday_invite_rank','qday_sametime_icount',
#0.8531  0.335  
#0.8172  0.363     798     0.02  0.015(qd day i rank)
#0.8409   0.3454  0.8085   0.0323 0.0046(u q sametime i)
         

In [15]:
#0.843927 0.813170  0.0307'uday_sametime_icount','uday_invite_rank','qday_invite_rank','qday_sametime_icount',
#0.820    0.8018    0.0181   'uday_sametime_icount','uday_invite_rank','qday_invite_rank'
#0.8172   0.798     0.02         (qd day i rank)
#0.8409   0.8085   0.0323 0.0046(u q sametime i)
#0.8531    0.335   
#0.847              0.81828      qitime_ratio稳定上0.005
#0.8502   0.33774   0.822 0.0282 today中关于q的特征
#0.8537    0.33477   0.811922 today中关于u的特征

In [14]:
test_id=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
test_id['target']=sub1
test_id.to_csv('result_8.txt',index=False, sep='\t',header=None)

In [15]:
0.8502-0.822

0.028200000000000003